In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import pickle
import pandas as pd
from transformers import GPT2TokenizerFast
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain import PromptTemplate, OpenAI, LLMChain


/home/carlos/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open('website_content_parsed.txt', 'r', encoding="utf-8") as f:
    text = f.read()

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

db = FAISS.from_documents(chunks, embeddings)

with open("embeddings.pkl", "wb") as f:
    pickle.dump(db, f)w

In [6]:
query = "request"
docs = db.similarity_search(query)
docs[0]

Document(page_content="GuideBelow you'll find examples using Fetch API but you can JSONPlaceholder with any other language.You can copy paste the code in your browser console to quickly test JSONPlaceholder.ads via CarbonGet 10 Free Images From Adobe Stock. Start Now.ADS VIA CARBONGetting a resourcefetch('https://jsonplaceholder.typicode.com/posts/1')  .then((response) => response.json())  .then((json) => console.log(json));👇 Output{  id: 1,  title: '...',  body: '...',  userId: 1}Listing all resourcesfetch('https://jsonplaceholder.typicode.com/posts')  .then((response) => response.json())  .then((json) => console.log(json));👇 Output[  { id: 1, title: '...' /* ... */ },  { id: 2, title: '...' /* ... */ },  { id: 3, title: '...' /* ... */ },  /* ... */  { id: 100, title: '...' /* ... */ },];Creating a resourcefetch('https://jsonplaceholder.typicode.com/posts', {  method: 'POST',  body: JSON.stringify({    title: 'foo',    body: 'bar',    userId: 1,  }),  headers: {    'Content-type': 'a

In [7]:
prompt_template = "create a openapi json for this documentation api {docs}?"

llm = OpenAI(temperature=0)
# llm_chain = LLMChain(
#     llm=llm,
#     prompt=PromptTemplate.from_template(prompt_template)
# )

llm("return me a openapi json for the page_content {}".format(docs))



'\n\n{\n    "openapi": "3.0.0",\n    "info": {\n        "title": "JSONPlaceholder API",\n        "version": "1.0.0"\n    },\n    "servers": [\n        {\n            "url": "https://jsonplaceholder.typicode.com"\n        }\n    ],\n    "paths": {\n        "/posts": {\n            "get": {\n                "summary": "Listing all resources",\n                "responses": {\n                    "200": {\n                        "description": "Successful response",\n                        "content": {\n                            "application/json": {\n                                "schema": {\n                                    "type": "array",\n                                    "items": {\n                                        "type": "object",\n                                        "properties": {\n                                            "id": {\n                                                "type": "integer"\n                                            },\n           